In [1]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
events = pd.read_csv('dataset/events_up_to_01062018.csv', low_memory = False)
test = pd.read_csv('dataset/labels_training_set.csv')

In [3]:
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#CREO UN DATA FRAME CON LAS PERSONAS PARA IR JOINEANDO MIS FEATURES AHI
persons = events['person'].to_frame().drop_duplicates()
persons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 2341657
Data columns (total 1 columns):
person    38829 non-null object
dtypes: object(1)
memory usage: 606.7+ KB


In [5]:
#ELIMINO COLUMNA MODEL YA QUE TENEMOS EL SKU QUE ES UN VALOR NUMERICO (MEJOR PARA LOS ALGORITMOS DE ML) REPRESENTANDO
#EL MODELO
events = events.drop(columns='model')

#TRANSFROMO LA FECHA A TIMESTAMP
events['timestamp'] = pd.to_datetime(events['timestamp'])

In [6]:
#AGREGO UNA COLUMNA DE CANTIDAD
events['cantidad'] = 1

In [7]:
events

,timestamp,event,person,url,sku,condition,storage,color,skus,search_term,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cantidad
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Excelente,32GB,Dourado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,Muito Bom,32GB,Cinza espacial,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,Muito Bom,64GB,Prateado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,Bom,128GB,Vermelho,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,Excelente,32GB,Branco,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,2018-05-18 00:44:27,searched products,4c8a8b93,NaN,NaN,NaN,NaN,NaN,"10240,9987,10322,10085,9944,9931,13404,10154,1...",iPhone se,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,2018-05-18 00:44:14,viewed product,1b9f7cf6,NaN,2831.0,Bom,16GB,Dourado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,2018-05-18 00:44:02,viewed product,29ebb414,NaN,2845.0,Bom,128GB,Cinza espacial,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,2018-05-18 00:43:59,viewed product,de8fe91b,NaN,12548.0,Bom,32GB,Platinum,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,2018-05-18 00:43:40,ad campaign hit,45baf068,/,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [8]:
#HACIENDO ESTE GROUPBY NOS QUEDAMOS CON LOS SKUS QUE VIO UNA PERSONA Y EN QUE FECHAg
grouped_sku = events.groupby(['person','sku']).agg({'cantidad':'sum'})
grouped_sku.sort_values(by='cantidad', ascending=False, inplace=True)
#grouped_sku.drop_duplicates(inplace=True, , keep='first')
grouped_sku = grouped_sku.reset_index().drop_duplicates(keep='first', subset='person')
grouped_sku.drop(columns='cantidad', inplace=True)
grouped_sku.columns = ['person', 'most_viewed_sku']
grouped_sku.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38250 entries, 0 to 622391
Data columns (total 2 columns):
person             38250 non-null object
most_viewed_sku    38250 non-null float64
dtypes: float64(1), object(1)
memory usage: 896.5+ KB


## DATAFRAME SKU MAS VISTO POR USUARIO

In [9]:
persons = pd.merge(persons, grouped_sku, how='left', on='person')
persons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Data columns (total 2 columns):
person             38829 non-null object
most_viewed_sku    38250 non-null float64
dtypes: float64(1), object(1)
memory usage: 910.1+ KB


In [13]:
persons.to_csv('features_gonza.csv')